## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.09,71,28,7.96,scattered clouds
1,1,Albany,US,42.6001,-73.9662,90.03,50,91,1.01,overcast clouds
2,2,Avarua,CK,-21.2078,-159.7750,71.65,88,100,11.50,light rain
3,3,New Norfolk,AU,-42.7826,147.0587,41.72,85,53,3.04,light rain
4,4,Hermanus,ZA,-34.4187,19.2345,56.70,73,5,9.60,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the lowest temperature you would like for your trip? "))
max_temp = float(input("What is the highest temperature you would like for your trip? "))

What is the lowest temperature you would like for your trip? 60
What is the highest temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
tempfilter_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                            (city_data_df["Max Temp"] >= min_temp)]
tempfilter_city_data_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.09,71,28,7.96,scattered clouds
2,2,Avarua,CK,-21.2078,-159.7750,71.65,88,100,11.50,light rain
6,6,Jamestown,US,42.0970,-79.2353,83.79,64,100,12.66,overcast clouds
8,8,Persianovskiy,RU,47.5137,40.1075,84.72,34,20,15.66,few clouds
9,9,Laguna,US,38.4210,-121.4238,86.94,50,0,5.75,clear sky
11,11,Lufilufi,WS,-13.8745,-171.5986,74.98,80,82,20.69,moderate rain
16,16,Kamina,CD,-8.7386,24.9906,67.39,38,16,5.14,few clouds
21,21,Sao Filipe,CV,14.8961,-24.4956,76.30,75,41,13.18,scattered clouds
22,22,Voh,NC,-20.9667,164.7000,68.23,87,61,5.01,broken clouds
23,23,Hilo,US,19.7297,-155.0900,78.28,93,100,1.01,overcast clouds


In [33]:
# 4a. Determine if there are any empty rows.
print(f"Null\n{tempfilter_city_data_df.isnull().sum()}\n")
print(f"Not Null\n{tempfilter_city_data_df.notnull().sum()}")

Null
City ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

Not Null
City ID                479
City                   479
Country                475
Lat                    479
Lng                    479
Max Temp               479
Humidity               479
Cloudiness             479
Wind Speed             479
Current Description    479
dtype: int64


In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = tempfilter_city_data_df.dropna()
print(f"Null\n{clean_preferred_cities_df.isnull().sum()}\n")
print(f"Not Null\n{clean_preferred_cities_df.notnull().sum()}")

Null
City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

Not Null
City ID                475
City                   475
Country                475
Lat                    475
Lng                    475
Max Temp               475
Humidity               475
Cloudiness             475
Wind Speed             475
Current Description    475
dtype: int64


In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.09,scattered clouds,-23.1203,-134.9692,
2,Avarua,CK,71.65,light rain,-21.2078,-159.7750,
6,Jamestown,US,83.79,overcast clouds,42.0970,-79.2353,
8,Persianovskiy,RU,84.72,few clouds,47.5137,40.1075,
9,Laguna,US,86.94,clear sky,38.4210,-121.4238,
11,Lufilufi,WS,74.98,moderate rain,-13.8745,-171.5986,
16,Kamina,CD,67.39,few clouds,-8.7386,24.9906,
21,Sao Filipe,CV,76.30,scattered clouds,14.8961,-24.4956,
22,Voh,NC,68.23,broken clouds,-20.9667,164.7000,
23,Hilo,US,78.28,overcast clouds,19.7297,-155.0900,


In [42]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame and add to params dictionary as values to the location key
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [48]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
print(clean_hotel_df.count())
clean_hotel_df.head(10)

City                   475
Country                475
Max Temp               475
Current Description    475
Lat                    475
Lng                    475
Hotel Name             475
dtype: int64


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.09,scattered clouds,-23.1203,-134.9692,People ThankYou
2,Avarua,CK,71.65,light rain,-21.2078,-159.7750,Paradise Inn
6,Jamestown,US,83.79,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
8,Persianovskiy,RU,84.72,few clouds,47.5137,40.1075,Otel' Dinastiya
9,Laguna,US,86.94,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
11,Lufilufi,WS,74.98,moderate rain,-13.8745,-171.5986,Leuaina Beach Resort and Spa
16,Kamina,CD,67.39,few clouds,-8.7386,24.9906,Guest House Florvia
21,Sao Filipe,CV,76.30,scattered clouds,14.8961,-24.4956,Hotel Xaguate
22,Voh,NC,68.23,broken clouds,-20.9667,164.7000,Le gîte du Koniambo
23,Hilo,US,78.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [50]:
# 8a. Create the output File (CSV)
vacation_output_csv = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(vacation_output_csv, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))